In [43]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

from Bio import SeqIO
from Bio.Data import IUPACData 
import csv
import numpy as np
import tensorflow as tf

#data_path = 'features_CENH3_DMR6_LUCA-CHLRE00002_orthologues.csv'
data_path = 'features_oma-seqs-viridiplantae_test.csv'

In [3]:
def protein2integer(in_seq):
    
    ## define universe of possible input values
    all_protein_letters = list(IUPACData.extended_protein_letters)
    #print(all_protein_letters)
    ## define a mapping of chars to integers
    char_to_int = dict((c, i) for i, c in enumerate(all_protein_letters))
    int_to_char = dict((i, c) for i, c in enumerate(all_protein_letters))
    ## integer encode input data
    integer_encoded = [char_to_int[char] for char in in_seq.upper()]
    
    #return(integer_encoded,len(all_protein_letters))
    return(integer_encoded)
    

In [4]:
def make_dataset(in_file):
    with open(in_file, 'r') as f:
        reader = csv.reader(f, delimiter="\t")
        # get all the rows as a list
        d_set = list(reader)
        # transform data into numpy array
        d_set = np.array(d_set).astype(str)
        
    integer_encoded_proteins = np.array([protein2integer(seq) for seq in d_set[:,1]])
    
    G = d_set[:, 0]
    X = integer_encoded_proteins
    Y = d_set[:, 2].astype(int)
                         
    return(d_set,G,X,Y)

In [16]:
def make_train_test_set(G,X,Y):
    indices = np.random.permutation(X.shape[0])
    train_size = int(indices.size*0.95)
    train_idx, test_idx = indices[:train_size], indices[train_size:]
    #print(len(train_idx),len(test_idx))
    
    X_train, X_test = X[train_idx,], X[test_idx,]
    #print(X_train.shape,X_test.shape)
    
    y_train, y_test = Y[train_idx,], Y[test_idx,]
    
    #print(X[train_idx[0],])
    #print(Y[train_idx[0],])

    #print(X_train[0,])
    #print(y_train[0,])
    
    return(X_train,y_train,X_test,y_test)

In [249]:
def model1(X_train_new, y_train,X_test_new, y_test):
    # https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
    # create the model
    embedding_vecor_length = 32
    model = Sequential()
    model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(100))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    model.fit(X_train_new, y_train, epochs=50, batch_size=100) #epochs=3, batch_size=64)
    # Final evaluation of the model
    scores = model.evaluate(X_test_new, y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    
    return()


In [259]:
def model2(X_train_new, y_train,X_test_new, y_test):
    # Initializing the Sequential model from KERAS.
    model = Sequential()

    max_review_length = len(max(X, key=len))

    # Creating a 16 neuron hidden layer with Linear Rectified activation function.
    #model.add(Dense(16, input_dim=1, init='uniform', activation='relu'))
    model.add(Dense(16, input_dim=max_review_length, kernel_initializer='uniform', activation='relu'))

    # Creating a 8 neuron hidden layer.
    model.add(Dense(8, kernel_initializer='uniform', activation='relu'))

    # Adding a output layer.
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    
    # Compiling the model
    model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

    print(model.summary())

    # Fitting the model
    #model.fit(X, Y, nb_epoch=150, batch_size=10)
    #scores = model.evaluate(X, Y)
    model.fit(X_train_new, y_train, epochs=1000, batch_size=100)
    scores = model.evaluate(X_test_new, y_test, verbose=0)

    print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))
    
    return()

In [80]:
def one_hot_matrix(labels,C):
    
    C = tf.constant(C,name="C")
    one_hot_matrix = tf.one_hot(labels,C,axis=1)
    sess = tf.Session()
    one_hot = sess.run(one_hot_matrix)
    sess.close()
    
    return one_hot

In [255]:
def model3(X_train_new, y_train,X_test_new, y_test):
    # Number of nodes in each NN hidden layer
    n_nodes_hl1 = 1500
    n_nodes_hl2 = 1500
    n_nodes_hl3 = 1500

    # Number of orthology clusters
    #n_classes = len(np.unique(np.concatenate((y_train,y_test),axis=0)))     #2 or 3 or ...
    
    #y_all = np.concatenate((y_train,y_test),axis=0)
    #y_min = np.amin(y_all)
    #n_classes = np.amax(y_all-y_min)+1
    n_classes = int(np.amax(np.concatenate((y_train,y_test),axis=0))+1)
    
    train_y = one_hot_matrix(y_train-y_min,n_classes)
    test_y = one_hot_matrix(y_test-y_min,n_classes)

    # Batch size and Epoch size for training the NN
    batch_size = 100   #100
    hm_epochs = 500    #1000

    # Initializing X and Y
    x = tf.placeholder('float')
    y = tf.placeholder('float')

    # Initializing NN layers
    hidden_1_layer = {'f_fum':n_nodes_hl1,
                  'weight':tf.Variable(tf.random_normal([len(X_train_new[0]), n_nodes_hl1])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'f_fum':n_nodes_hl2,
                  'weight':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'f_fum':n_nodes_hl3,
                  'weight':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'f_fum':None,
                'weight':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                'bias':tf.Variable(tf.random_normal([n_classes])),}


    
    
    l1 = tf.add(tf.matmul(x,hidden_1_layer['weight']), hidden_1_layer['bias'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weight']), hidden_2_layer['bias'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weight']), hidden_3_layer['bias'])
    l3 = tf.nn.relu(l3)

    prediction = tf.matmul(l3,output_layer['weight']) + output_layer['bias']

        
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=y) )
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        #try:
        #    epoch = int(open(tf_log,'r').read().split('\n')[-2])+1
        #    print('STARTING:',epoch)
        #except:
        #    epoch = 1
        epoch = 1

        while epoch <= hm_epochs:
            epoch_loss = 1
            
            i=0
            while i < len(X_train_new):
                start = i
                end = i+batch_size
                batch_x = np.array(X_train_new[start:end])
                batch_y = np.array(train_y[start:end])

                _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,y: batch_y})
                epoch_loss += c
                i+=batch_size
                
            
            print('Epoch ',epoch,'/',hm_epochs,'- loss:',epoch_loss)
 
            
            #with open(tf_log,'a') as f:
            #    f.write(str(epoch)+'\n') 
            epoch +=1
            
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

        #print("\nModel saved in path: %s " % my_model_save_path)
        print('\nAccuracy:',accuracy.eval({x:X_test_new, y:test_y}))
    return()

In [8]:
def test_1(d_set):
    print(d_set[:,0])
    print(d_set[0:2,1])
    print(d_set[:,2].astype(int))
    print(d_set.shape)
    print(d_set[:,1].shape)
    return()

In [9]:
def test_2(d_set):
    integer_encoded_proteins = np.array([protein2integer(seq) for seq in d_set[:,1]])
    print(len(integer_encoded_proteins))
    print(integer_encoded_proteins[0])
    #np.array(integer_encoded_proteins).shape
    print(integer_encoded_proteins.shape)
    #protein2integer(dataset[:,1])
    return()

In [10]:
def test_3(G,X,Y):
    print(G.shape)
    print(X.shape)
    print(Y.shape)

    print(G[0:3,])
    print(X[0:3,])
    print(Y[0:3,])
    
    print('Maximum review length: {}'.format(len(max(X, key=len))))
    print('Minimum review length: {}'.format(len(min(X, key=len))))
    
    return()

In [11]:
def test_4(X_train_new,X_train):
    print(X_train_new.shape)
    print(X_train_new[0,:])
    print(X_train.shape)
    print(X_train[0,])
    return()


In [17]:
dataset, G, X, Y = make_dataset(data_path)
X_train,y_train,X_test,y_test = make_train_test_set(G,X,Y)

#print("============ Test 1 =======================")
#test_1(dataset)
#print("============ Test 2 =======================")
#test_2(dataset)
#print("============ Test 3 =======================")
#test_3(G,X,Y)



In [18]:
top_words = len(list(IUPACData.extended_protein_letters)) # = 26
max_review_length = len(max(X, key=len)) # = 500
# truncate and pad input sequences
X_train_new = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test_new = sequence.pad_sequences(X_test, maxlen=max_review_length)
  
#print("============ Test 4 =======================")
#test_4(X_train_new,X_train)


In [260]:
model2(X_train_new, y_train,X_test_new, y_test)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 16)                72064     
_________________________________________________________________
dense_27 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 9         
Total params: 72,209
Trainable params: 72,209
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000
3217/3217 [==============================] - 1s 287us/step - loss: -13970.2898 - acc: 0.0000e+00
Epoch 2/1000
3217/3217 [==============================] - 0s 43us/step - loss: -61507.6485 - acc: 0.0000e+00
Epoch 3/1000
3217/3217 [==============================] - 0s 45us/step - loss: -66066.7258 - acc: 0.0000e+00
Epoch 4/1000
3217/3217 [==============================] - 0s 45us/ste

3217/3217 [==============================] - 0s 46us/step - loss: -66360.2177 - acc: 0.0000e+00
Epoch 69/1000
3217/3217 [==============================] - 0s 43us/step - loss: -66360.2183 - acc: 0.0000e+00
Epoch 70/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2173 - acc: 0.0000e+00
Epoch 71/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2182 - acc: 0.0000e+00
Epoch 72/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2167 - acc: 0.0000e+00
Epoch 73/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2178 - acc: 0.0000e+00
Epoch 74/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2179 - acc: 0.0000e+00
Epoch 75/1000
3217/3217 [==============================] - 0s 42us/step - loss: -66360.2173 - acc: 0.0000e+00
Epoch 76/1000
3217/3217 [==============================] - 0s 42us/step - loss: -66360.2185 - acc: 0.0000e+00
Epoch 77/1000
3217/3217 

3217/3217 [==============================] - 0s 43us/step - loss: -66360.2176 - acc: 0.0000e+00
Epoch 143/1000
3217/3217 [==============================] - 0s 42us/step - loss: -66360.2172 - acc: 0.0000e+00
Epoch 144/1000
3217/3217 [==============================] - 0s 42us/step - loss: -66360.2172 - acc: 0.0000e+00
Epoch 145/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2176 - acc: 0.0000e+00
Epoch 146/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2169 - acc: 0.0000e+00
Epoch 147/1000
3217/3217 [==============================] - 0s 45us/step - loss: -66360.2176 - acc: 0.0000e+00
Epoch 148/1000
3217/3217 [==============================] - 0s 43us/step - loss: -66360.2167 - acc: 0.0000e+00
Epoch 149/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2172 - acc: 0.0000e+00
Epoch 150/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2169 - acc: 0.0000e+00
Epoch 151/1000
3

3217/3217 [==============================] - 0s 43us/step - loss: -66360.2180 - acc: 0.0000e+00
Epoch 216/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2165 - acc: 0.0000e+00
Epoch 217/1000
3217/3217 [==============================] - 0s 42us/step - loss: -66360.2166 - acc: 0.0000e+00
Epoch 218/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2161 - acc: 0.0000e+00
Epoch 219/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2174 - acc: 0.0000e+00
Epoch 220/1000
3217/3217 [==============================] - 0s 42us/step - loss: -66360.2172 - acc: 0.0000e+00
Epoch 221/1000
3217/3217 [==============================] - 0s 40us/step - loss: -66360.2174 - acc: 0.0000e+00
Epoch 222/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2181 - acc: 0.0000e+00
Epoch 223/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2179 - acc: 0.0000e+00
Epoch 224/1000
3

3217/3217 [==============================] - 0s 46us/step - loss: -66360.2165 - acc: 0.0000e+00
Epoch 289/1000
3217/3217 [==============================] - 0s 43us/step - loss: -66360.2164 - acc: 0.0000e+00
Epoch 290/1000
3217/3217 [==============================] - 0s 42us/step - loss: -66360.2181 - acc: 0.0000e+00
Epoch 291/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2177 - acc: 0.0000e+00
Epoch 292/1000
3217/3217 [==============================] - 0s 57us/step - loss: -66360.2170 - acc: 0.0000e+00
Epoch 293/1000
3217/3217 [==============================] - 0s 47us/step - loss: -66360.2175 - acc: 0.0000e+00
Epoch 294/1000
3217/3217 [==============================] - 0s 44us/step - loss: -66360.2176 - acc: 0.0000e+00
Epoch 295/1000
3217/3217 [==============================] - 0s 43us/step - loss: -66360.2183 - acc: 0.0000e+00
Epoch 296/1000
3217/3217 [==============================] - 0s 43us/step - loss: -66360.2168 - acc: 0.0000e+00
Epoch 297/1000
3

...

3217/3217 [==============================] - 0s 41us/step - loss: -66360.2173 - acc: 0.0000e+00
Epoch 727/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2177 - acc: 0.0000e+00
Epoch 728/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2172 - acc: 0.0000e+00
Epoch 729/1000
3217/3217 [==============================] - 0s 42us/step - loss: -66360.2163 - acc: 0.0000e+00
Epoch 730/1000
3217/3217 [==============================] - 0s 42us/step - loss: -66360.2181 - acc: 0.0000e+00
Epoch 731/1000
3217/3217 [==============================] - 0s 42us/step - loss: -66360.2169 - acc: 0.0000e+00
Epoch 732/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2163 - acc: 0.0000e+00
Epoch 733/1000
3217/3217 [==============================] - 0s 42us/step - loss: -66360.2166 - acc: 0.0000e+00
Epoch 734/1000
3217/3217 [==============================] - 0s 42us/step - loss: -66360.2162 - acc: 0.0000e+00
Epoch 735/1000
3

3217/3217 [==============================] - 0s 43us/step - loss: -66360.2173 - acc: 0.0000e+00
Epoch 800/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2172 - acc: 0.0000e+00
Epoch 801/1000
3217/3217 [==============================] - 0s 42us/step - loss: -66360.2169 - acc: 0.0000e+00
Epoch 802/1000
3217/3217 [==============================] - 0s 42us/step - loss: -66360.2170 - acc: 0.0000e+00
Epoch 803/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2177 - acc: 0.0000e+00
Epoch 804/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2182 - acc: 0.0000e+00
Epoch 805/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2174 - acc: 0.0000e+00
Epoch 806/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2170 - acc: 0.0000e+00
Epoch 807/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2183 - acc: 0.0000e+00
Epoch 808/1000
3

3217/3217 [==============================] - 0s 42us/step - loss: -66360.2168 - acc: 0.0000e+00
Epoch 873/1000
3217/3217 [==============================] - 0s 42us/step - loss: -66360.2172 - acc: 0.0000e+00
Epoch 874/1000
3217/3217 [==============================] - 0s 42us/step - loss: -66360.2174 - acc: 0.0000e+00
Epoch 875/1000
3217/3217 [==============================] - 0s 42us/step - loss: -66360.2174 - acc: 0.0000e+00
Epoch 876/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2177 - acc: 0.0000e+00
Epoch 877/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2165 - acc: 0.0000e+00
Epoch 878/1000
3217/3217 [==============================] - 0s 42us/step - loss: -66360.2169 - acc: 0.0000e+00
Epoch 879/1000
3217/3217 [==============================] - 0s 42us/step - loss: -66360.2161 - acc: 0.0000e+00
Epoch 880/1000
3217/3217 [==============================] - 0s 42us/step - loss: -66360.2172 - acc: 0.0000e+00
Epoch 881/1000
3

3217/3217 [==============================] - 0s 42us/step - loss: -66360.2180 - acc: 0.0000e+00
Epoch 946/1000
3217/3217 [==============================] - 0s 43us/step - loss: -66360.2179 - acc: 0.0000e+00
Epoch 947/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2174 - acc: 0.0000e+00
Epoch 948/1000
3217/3217 [==============================] - 0s 42us/step - loss: -66360.2166 - acc: 0.0000e+00
Epoch 949/1000
3217/3217 [==============================] - 0s 43us/step - loss: -66360.2175 - acc: 0.0000e+00
Epoch 950/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2172 - acc: 0.0000e+00
Epoch 951/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2168 - acc: 0.0000e+00
Epoch 952/1000
3217/3217 [==============================] - 0s 41us/step - loss: -66360.2177 - acc: 0.0000e+00
Epoch 953/1000
3217/3217 [==============================] - 0s 42us/step - loss: -66360.2170 - acc: 0.0000e+00
Epoch 954/1000
3

()

In [23]:
model1(X_train_new, y_train,X_test_new, y_test)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4503, 32)          832       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 4503, 32)          3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 2251, 32)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 101       
Total params: 57,237
Trainable params: 57,237
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
3217/3217 [==============================] - 94s 29ms/step - loss: -41235.7428 - acc: 0.0000e+00
Epoch 2/10
3217/3217 [===

()

In [253]:
model3(X_train_new,y_train,X_test_new,y_test)

('Epoch', 1, 'completed out of', 1000, 'loss:', 355915819.0)
('Epoch', 2, 'completed out of', 1000, 'loss:', 167514745.5)
('Epoch', 3, 'completed out of', 1000, 'loss:', 106282263.625)
('Epoch', 4, 'completed out of', 1000, 'loss:', 75404862.96875)
('Epoch', 5, 'completed out of', 1000, 'loss:', 56485900.359375)
('Epoch', 6, 'completed out of', 1000, 'loss:', 43748544.087890625)
('Epoch', 7, 'completed out of', 1000, 'loss:', 34602131.775634766)
('Epoch', 8, 'completed out of', 1000, 'loss:', 27685861.0546875)
('Epoch', 9, 'completed out of', 1000, 'loss:', 22524449.768310547)
('Epoch', 10, 'completed out of', 1000, 'loss:', 18256621.25)
('Epoch', 11, 'completed out of', 1000, 'loss:', 14801742.810661316)
('Epoch', 12, 'completed out of', 1000, 'loss:', 12029841.134277344)
('Epoch', 13, 'completed out of', 1000, 'loss:', 9595841.953125)
('Epoch', 14, 'completed out of', 1000, 'loss:', 7806541.737182617)
('Epoch', 15, 'completed out of', 1000, 'loss:', 6209142.296875)
('Epoch', 16, 'com

('Epoch', 122, 'completed out of', 1000, 'loss:', 76239.53218078613)
('Epoch', 123, 'completed out of', 1000, 'loss:', 47851.15444946289)
('Epoch', 124, 'completed out of', 1000, 'loss:', 20243.28269958496)
('Epoch', 125, 'completed out of', 1000, 'loss:', 7875.833690643311)
('Epoch', 126, 'completed out of', 1000, 'loss:', 6607.001373291016)
('Epoch', 127, 'completed out of', 1000, 'loss:', 4570.487571716309)
('Epoch', 128, 'completed out of', 1000, 'loss:', 5950.127513885498)
('Epoch', 129, 'completed out of', 1000, 'loss:', 3501.793727874756)
('Epoch', 130, 'completed out of', 1000, 'loss:', 3392.8150482177734)
('Epoch', 131, 'completed out of', 1000, 'loss:', 5503.275944326248)
('Epoch', 132, 'completed out of', 1000, 'loss:', 5013.841197013855)
('Epoch', 133, 'completed out of', 1000, 'loss:', 1984.8749694824219)
('Epoch', 134, 'completed out of', 1000, 'loss:', 6863.056179046631)
('Epoch', 135, 'completed out of', 1000, 'loss:', 581.6762504577637)
('Epoch', 136, 'completed out of

('Epoch', 241, 'completed out of', 1000, 'loss:', 6666.439963340759)
('Epoch', 242, 'completed out of', 1000, 'loss:', 5063.977456569672)
('Epoch', 243, 'completed out of', 1000, 'loss:', 4484.141258239746)
('Epoch', 244, 'completed out of', 1000, 'loss:', 9893.591342926025)
('Epoch', 245, 'completed out of', 1000, 'loss:', 3475.5549545288086)
('Epoch', 246, 'completed out of', 1000, 'loss:', 14105.676406860352)
('Epoch', 247, 'completed out of', 1000, 'loss:', 16806.20360183716)
('Epoch', 248, 'completed out of', 1000, 'loss:', 16310.098915100098)
('Epoch', 249, 'completed out of', 1000, 'loss:', 4325.186233520508)
('Epoch', 250, 'completed out of', 1000, 'loss:', 7872.418685913086)
('Epoch', 251, 'completed out of', 1000, 'loss:', 3795.8512115478516)
('Epoch', 252, 'completed out of', 1000, 'loss:', 7408.362487792969)
('Epoch', 253, 'completed out of', 1000, 'loss:', 6504.9311447143555)
('Epoch', 254, 'completed out of', 1000, 'loss:', 40963.25845336914)
('Epoch', 255, 'completed out

('Epoch', 362, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 363, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 364, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 365, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 366, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 367, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 368, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 369, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 370, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 371, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 372, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 373, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 374, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 375, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 376, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 377, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 378, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 379, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 

('Epoch', 511, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 512, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 513, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 514, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 515, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 516, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 517, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 518, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 519, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 520, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 521, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 522, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 523, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 524, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 525, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 526, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 527, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 528, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 

('Epoch', 660, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 661, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 662, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 663, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 664, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 665, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 666, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 667, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 668, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 669, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 670, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 671, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 672, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 673, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 674, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 675, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 676, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 677, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 

('Epoch', 809, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 810, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 811, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 812, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 813, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 814, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 815, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 816, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 817, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 818, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 819, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 820, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 821, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 822, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 823, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 824, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 825, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 826, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 

('Epoch', 958, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 959, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 960, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 961, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 962, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 963, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 964, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 965, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 966, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 967, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 968, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 969, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 970, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 971, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 972, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 973, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 974, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 975, 'completed out of', 1000, 'loss:', 1.0)
('Epoch', 

()

In [70]:
print(X_train_new.shape)
print(X_train_new.flatten().shape)
print(len(X_train_new))

(3217, 4503)
(14486151,)
3217


In [174]:
print(np.array(X_train_new[0:10]).shape)
print(y_train[0])
x = np.array(one_hot_matrix(y_train[0:10],len(np.unique(y_train))))
print(x)
print(np.array(y_train[0:10]))
print(np.array(y_train).shape)
print(np.array(y_train).reshape((1, 3217)).shape)
print(np.amax(y_train[0:10])+1)
print(np.array(one_hot_matrix(y_train[0:10],6993)).shape)

(10, 4503)
2411
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[2411 3756 2775 3445 1558 1945 2222 3949 1153 6992]
(3217,)
(1, 3217)
6993
(10, 6993)


In [218]:
testarr = np.zeros(([4,]))
testarr[0:4]=(5,4,2,3)
print(testarr)
new = np.zeros(([4,6]))
print(testarr-np.amin(testarr))
print(new)
n=0
for i in testarr: 
    print(i)
    #i -= 1
    new[int(n),int(i)]=1
    n += 1
    
print(new)

print(one_hot_matrix(testarr,6).T)
new.shape

[5. 4. 2. 3.]
[3. 2. 0. 1.]
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
5.0
4.0
2.0
3.0
[[0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]


(4, 6)

In [205]:
print(np.amax(np.concatenate((y_train,y_test),axis=0)))

8220


In [236]:
y_all = np.concatenate((y_train,y_test),axis=0)
y_min = np.amin(y_all)
print(y_min)
n_classes = np.amax(y_all-y_min)+1
print(n_classes)
train_y = one_hot_matrix(y_train,int(n_classes))

6
8215
